# Instructions

1. Paste the token copied to your clipboard provided from the 'Cluster' Workflow dashboard.
2. Click the ▶️  on the left or go to "Runtime" -> "Run All" and click "Run anyway" on the warning that pops up.
3. You should see a progress bar underneath the form, keep this window opened and active until the progress bar is complete otherwise it'll terminate.

Note: 
- For fastest clustering speed make sure to go to "Runtime" -> "Change runtime type" and enable "Hardware accelerator" as "GPU".

In [ ]:
#@title Paste token below and press ▶️  button to the left of this title { display-mode: "form" }
# %tb

token = "" #@param {type:"string"}


show_warnings_in_logs = False #@param {type:"boolean"}
#@markdown Once the form is filled and you've clicked run, monitor below for logs of it running

import base64
import json
import warnings
warnings.filterwarnings('ignore')

config = json.loads(base64.b64decode(token + "==="))

import subprocess

def install_package(package):
    process = subprocess.Popen(['pip', 'install', package],
                        stdout=subprocess.PIPE, 
                        stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    return
print(config)

!pip install -q RelevanceAI==0.33.2 --quiet
print("Installing RelevanceAI")

import contextlib

class DevNull:
    def write(self, msg):
        pass


## Instantiate client ###
from relevanceai import Client 
if config['region'] != 'old-australia-east':
  region = config['region']
  base_url = f"https://api.{region}.relevance.ai/latest/"
  client = Client(config['project'], config['api_key'])
  client.base_url = base_url
  client.ingest_base_url = base_url
  client.config['api.base_url'] = client.base_url
  client.config['api.base_ingest_url'] = client.base_url
else:
  client = Client(config['project'], config['api_key'])

## Checking valid vector field ###
for v in config['vector_fields']:
  if not '_vector_'in v:
    raise ValueError(f"'{v}' is not a valid vector field")


## Instantiate client ###
try:

  dataset_id = config['dataset_id']
  df = client.Dataset(dataset_id)
  
  cluster_method = 'kmeans'
  if df.shape[0] > 3000:
    cluster_method = 'minibatchkmeans'

  clusterer = df.auto_cluster(f"{cluster_method}-{config['n_clusters']}", vector_fields=config['vector_fields'])

except Exception as e:
    raise ValueError('Incorrect token provided')


print("Finished clustering your data, you may close this window.")